<a href="https://colab.research.google.com/github/JinalkaHerath/Greenhouse-AI-Fan-Controller/blob/main/Greenhouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf


In [ ]:
# Load the dataset
df = pd.read_csv("/content/sample_data/grenhouse.csv")

# Drop the timestamp column
df = df.drop(columns=["timestamp"])

# Split features and label
X = df[["temperature_c", "humidity_percent"]]
y = df["high_temp_status"]

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
# Build DNN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/50
395/395 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8502 - loss: 0.3856 - val_accuracy: 0.9579 - val_loss: 0.1272
Epoch 2/50
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9584 - loss: 0.1012 - val_accuracy: 0.9836 - val_loss: 0.0647
Epoch 3/50
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9854 - loss: 0.0567 - val_accuracy: 0.9964 - val_loss: 0.0465
Epoch 4/50
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9876 - loss: 0.0426 - val_accuracy: 0.9957 - val_loss: 0.0366
Epoch 5/50
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9906 - loss: 0.0327 - val_accuracy: 0.9971 - val_loss: 0.0309
Epoch 6/50
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9920 - loss: 0.0285 - val_accuracy: 0.9957 - val_loss: 0.0279
Epoch 7/50
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9939 - loss: 0.0243 - val_accuracy: 0.9886 - val_loss: 0.0219
Epoch 8/50
395/395 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9942 - loss: 0.0218 - val_accuracy: 0.

In [ ]:
# Evaluate on test set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

# Print accuracy and classification report
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9971469329529244
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      1565
           1       0.99      1.00      1.00      1940

    accuracy                           1.00      3505
   macro avg       1.00      1.00      1.00      3505
weighted avg       1.00      1.00      1.00      3505



In [ ]:
# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save model
with open("fan_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as fan_model.tflite")


Saved artifact at '/tmp/tmpy9umb9w7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139878988581584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139879256799760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139878988579856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139878988579472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139878988583312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139878988583120: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model saved as fan_model.tflite


In [ ]:
from google.colab import files
files.download("fan_model.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Predict a custom sample
sample = np.array([[25.0, 85.0]])  # Example: temp=25°C, humidity=85%
sample_scaled = scaler.transform(sample)
prediction = model.predict(sample_scaled)
status = "ON" if prediction > 0.5 else "OFF"
print("Fan Status Prediction:", status)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Fan Status Prediction: OFF


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
